In [1]:
%matplotlib inline
import os
os.environ['PY3_PROD'] = '1'
%load_ext autoreload
%autoreload 2
os.system('kinit')

1

In [3]:
import numpy as np
import math
import pandas as pd
from typing import Union, List
import datetime
from pycmqlib3.utility import misc
from pycmqlib3.utility.dbaccess import *
import pycmqlib3.analytics.data_handler as dh
import pathlib
from pycmqlib3.utility.process_wt_data import *

# from pycmqlib3.utility import dataseries
# from pycmqlib3.analytics.tstool import *
# from pycmqlib3.analytics.btmetrics import *


In [ ]:
base_folder = "C:/dev/wtdev/storage/his"

zip_wt_dir(base_folder, 'C:/dev/data/wt_data.zip', cutoff=datetime.date(2022,10,22))

In [ ]:
base_folder = "C:/dev/wtdev/storage/his"
update_folder = "C:/dev/data/his"
cutoff = datetime.date(2022,10,22)
update_wt_store(base_folder=base_folder, update_folder = update_folder, cutoff=cutoff)

In [4]:
pd.set_option('display.max_rows', 150)
pd.set_option('display.max_columns', 150)

In [ ]:
start_date = datetime.date(2000,1,1)
end_date = datetime.date(2022,10,14)
src_folder = 'c:/dev/data/his'
dst_folder = 'c:/dev/data/his_new'
cutoff_date = 20221001

save_bars_to_wt_store(exchange_list=['DCE', 'CZCE', 'SHFE', 'INE', 'CFFEX'],
                        start_date=start_date,
                        end_date=end_date,
                        cutoff_date=cutoff_date,
                        process_min=True,
                        process_day=True,
                        src_folder=src_folder,
                        dst_folder=dst_folder)

In [14]:
sdate = datetime.date(2000,1,1)
edate = datetime.date(2022,9,30)

cnx = connect(**dbconfig)
cont = 'TA301'
mdf = load_min_data_to_df(cnx, 'fut_min', cont, sdate, edate, index_col = None, shift_datetime = True)

# mdf['rdate'] = mdf['date'].astype('datetime64').dt.strftime('%Y%m%d').astype('int64')
# mdf['time'] = 0
# mdf['hold'] = mdf['openInterest']
# mdf['diff'] = mdf['hold'].diff().fillna(0)
# mdf['money'] = mdf['close'] * mdf['volume']
# mdf['vol'] = mdf['volume']
col_list = ['date', 'time', 'open', 'high', 'low', 'close', 'money', 'vol', 'hold', 'diff']
# mdf = mdf[col_list]            
mdf = conv_min_data(mdf)
print(mdf[col_list])

           date        time    open    high     low   close     money  vol  \
0      20120120  2201201015  8780.0  8780.0  8780.0  8780.0   17560.0    2   
1      20120130  2201300901  8824.0  8824.0  8824.0  8824.0   17648.0    2   
2      20120131  2201311008  9016.0  9016.0  9016.0  9016.0   18032.0    2   
3      20120131  2201311046  9016.0  9070.0  9016.0  9070.0   18140.0    2   
4      20120131  2201311055  9070.0  9070.0  9066.0  9066.0   18132.0    2   
...         ...         ...     ...     ...     ...     ...       ...  ...   
38845  20130115  2301151001  8874.0  9000.0  8874.0  9000.0   36000.0    4   
38846  20130115  2301151103  9000.0  9022.0  9000.0  9022.0   36088.0    4   
38847  20130116  2301161500  9022.0  9022.0  8850.0  8850.0  141600.0   16   
38848  20130117  2301170929  9078.0  9078.0  9078.0  9078.0  108936.0   12   
38849  20130117  2301171037  9078.0  9078.0  8906.0  8906.0   35624.0    4   

       hold   diff  
0         2    0.0  
1         2    0.0  


In [31]:
ddf = load_daily_data_to_df(cnx, 'fut_daily', cont, datetime.date(2000,1,1), edate, index_col='date', field = 'instID', date_as_str = False)
ddf = ddf.rename(columns = {'openInterest': 'open_interest'}).reset_index()
ddf['pre_settle'] = ddf['settle'].shift(1)
ddf['pre_interest'] = ddf['open_interest'].shift(1)
ddf['pre_close'] = ddf['close'].shift(1)
ddf['date'] = ddf['date'].astype('datetime64').dt.strftime('%Y%m%d').astype('int64')
ddf = ddf.set_index('date')

In [1]:
ddf

NameError: name 'ddf' is not defined

In [10]:
cnx = connect(**dbconfig)
cont = 'c2211'
mdf = load_min_data_to_df(cnx, 'fut_min', cont, start_date, end_date, index_col=None, shift_datetime=False)
mdf['multiple'] = 10
mdf2 = conv_min_data(mdf)
mdf2

In [11]:
period = 'min1'
exch = 'DCE'
cont = 'a2301'
src_folder = 'c:/dev/wtdev/storage/his'
filename = '%s/%s/%s/%s.dsb' % (src_folder, period, exch, cont)
dtHelper = WtDataHelper()
curr_df = dtHelper.read_dsb_bars(filename).to_df()
curr_df

,date,bartime,open,high,low,close,settle,money,volume,hold,diff
202201172116,20220118,202201172116,5805.0,5805.0,5805.0,5805.0,0.0,58050.0,1.0,1.0,0.0
202201172259,20220118,202201172259,5805.0,5805.0,5805.0,5805.0,0.0,58050.0,1.0,2.0,1.0
202201180906,20220118,202201180906,5805.0,5808.0,5805.0,5808.0,0.0,58080.0,1.0,2.0,0.0
202201180908,20220118,202201180908,5808.0,5808.0,5799.0,5800.0,0.0,116000.0,2.0,4.0,2.0
202201180914,20220118,202201180914,5808.0,5808.0,5779.0,5779.0,0.0,115580.0,2.0,6.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...
202211101456,20221110,202211101456,5687.0,5687.0,5683.0,5685.0,0.0,60939000.0,1072.0,113066.0,-426.0
202211101457,20221110,202211101457,5685.0,5685.0,5680.0,5683.0,0.0,67846080.0,1194.0,113006.0,-60.0
202211101458,20221110,202211101458,5684.0,5684.0,5680.0,5683.0,0.0,48355110.0,851.0,112770.0,-236.0
202211101459,20221110,202211101459,5683.0,5685.0,5682.0,5685.0,0.0,26260760.0,462.0,112618.0,-152.0


In [6]:
from pycmqlib3.analytics import data_handler as dh

In [13]:
curr_df['min_id'] = curr_df['bartime'].apply(lambda x: wt_time_to_min_id(x%10000))
curr_df

,date,bartime,open,high,low,close,settle,money,volume,hold,diff,min_id
202201172116,20220118,202201172116,5805.0,5805.0,5805.0,5805.0,0.0,58050.0,1.0,1.0,0.0,316
202201172259,20220118,202201172259,5805.0,5805.0,5805.0,5805.0,0.0,58050.0,1.0,2.0,1.0,459
202201180906,20220118,202201180906,5805.0,5808.0,5805.0,5808.0,0.0,58080.0,1.0,2.0,0.0,1506
202201180908,20220118,202201180908,5808.0,5808.0,5799.0,5800.0,0.0,116000.0,2.0,4.0,2.0,1508
202201180914,20220118,202201180914,5808.0,5808.0,5779.0,5779.0,0.0,115580.0,2.0,6.0,2.0,1514
...,...,...,...,...,...,...,...,...,...,...,...,...
202211101456,20221110,202211101456,5687.0,5687.0,5683.0,5685.0,0.0,60939000.0,1072.0,113066.0,-426.0,2056
202211101457,20221110,202211101457,5685.0,5685.0,5680.0,5683.0,0.0,67846080.0,1194.0,113006.0,-60.0,2057
202211101458,20221110,202211101458,5684.0,5684.0,5680.0,5683.0,0.0,48355110.0,851.0,112770.0,-236.0,2058
202211101459,20221110,202211101459,5683.0,5685.0,5682.0,5685.0,0.0,26260760.0,462.0,112618.0,-152.0,2059


In [19]:
curr_df.high.values[-3:]

array([5684., 5685., 5694.])

In [35]:
if not None: print('1')

1


In [26]:
import zipfile

In [39]:
def zipdir(path, filename, cutoff = None, file_type = '.dsb'):
    with zipfile.ZipFile(filename, 'w', zipfile.ZIP_DEFLATED) as ziph:
        for root, dirs, files in os.walk(path):
            for file in files:
                if file.endswith(file_type):
                    time_mod = os.path.getmtime(f'{root}/{file}')
                    time_mod = datetime.datetime.fromtimestamp(time_mod)
                    if cutoff and (time_mod.date() < cutoff):
                        continue
                    ziph.write(os.path.join(root, file), os.path.relpath(os.path.join(root, file), os.path.join(path, '..')))

cutoff = datetime.date(2022,11,5)
zipdir('c:/dev/wtdev/storage/his/', 'C:/dev/data/test_zip.zip', cutoff = cutoff)

In [ ]:
import os
import zipfile
    
def zipdir(path, ziph):
    # ziph is zipfile handle
    for root, dirs, files in os.walk(path):
        for file in files:
            if file.endswith('.dsb'):
                time_mod = os.path.getmtime('{root}/{file}')
                time_mod = datetime.datetime.fromtimestamp(time_mod).strftime('%Y-%m-%d %H:%M:%S')
                print(file, time_mod)
                
            ziph.write(os.path.join(root, file), os.path.relpath(os.path.join(root, file), os.path.join(path, '..')))

with zipfile.ZipFile('Python.zip', 'w', zipfile.ZIP_DEFLATED) as zipf:
    zipdir('tmp/', zipf)

In [ ]:
from pathlib import Path
Path("/my/directory").mkdir(parents=True, exist_ok=True)

In [25]:
src_folder = 'c:/dev/wtdev/storage/his/'
dst_folder =
for period in ['min1', 'min5', 'day']:
    for exch in ['CFFEX', 'DCE', "CZCE", 'SHFE', "INE"]:
        for file in os.listdir(f'{src_folder}/{period}/{exch}/'):
            if file.endswith('.dsb'):
                time_mod = os.path.getmtime('c:/dev/wtdev/storage/his/min1/CFFEX/' + file)
                time_mod = datetime.datetime.fromtimestamp(time_mod).strftime('%Y-%m-%d %H:%M:%S')
                print(file, time_mod)

IC1505.dsb 2022-10-25 01:00:59
IC1506.dsb 2022-10-25 01:00:59
IC1507.dsb 2022-10-25 01:00:59
IC1508.dsb 2022-10-25 01:00:59
IC1509.dsb 2022-10-25 01:01:00
IC1510.dsb 2022-10-25 01:01:00
IC1511.dsb 2022-10-25 01:01:00
IC1512.dsb 2022-10-25 01:01:00
IC1601.dsb 2022-10-25 01:01:00
IC1602.dsb 2022-10-25 01:01:00
IC1603.dsb 2022-10-25 01:01:01
IC1604.dsb 2022-10-25 01:01:01
IC1605.dsb 2022-10-25 01:01:01
IC1606.dsb 2022-10-25 01:01:01
IC1607.dsb 2022-10-25 01:01:01
IC1608.dsb 2022-10-25 01:01:01
IC1609.dsb 2022-10-25 01:01:01
IC1610.dsb 2022-10-25 01:01:02
IC1611.dsb 2022-10-25 01:01:02
IC1612.dsb 2022-10-25 01:01:02
IC1701.dsb 2022-10-25 01:01:02
IC1702.dsb 2022-10-25 01:01:02
IC1703.dsb 2022-10-25 01:01:02
IC1704.dsb 2022-10-25 01:01:02
IC1705.dsb 2022-10-25 01:01:03
IC1706.dsb 2022-10-25 01:01:03
IC1707.dsb 2022-10-25 01:01:03
IC1708.dsb 2022-10-25 01:01:03
IC1709.dsb 2022-10-25 01:01:03
IC1710.dsb 2022-10-25 01:01:03
IC1711.dsb 2022-10-25 01:01:03
IC1712.dsb 2022-10-25 01:01:04
IC1801.d

In [44]:
src_folder = 'c:/dev/data/his/ticks/'
dst_folder = 'c:/dev/data/his_221021/ticks/'

In [45]:
import shutil
shutil.copytree(src_folder, dst_folder, dirs_exist_ok=True)

'c:/dev/data/his_221021/ticks/'